In [1]:
using CSV
using DataFrames
using Dates
using Gadfly

In [2]:
ENV["COLUMNS"] = 1000;

In [3]:
rawdata = DataFrame(CSV.File("../rawdata/reported_hospital_capacity_admissions_facility_level_weekly_average_timeseries_20201221_0.csv"));
data = select(rawdata,
    :hospital_name => ByRow(titlecase) => :hospital,
    :hospital_pk => :hospital_id,
    :collection_week => :date,
    :all_adult_hospital_inpatient_beds_7_day_avg => ByRow(x -> (ismissing(x) || x == -999999) ? missing : x) => :beds_allbeds,
    :total_icu_beds_7_day_avg => ByRow(x -> (ismissing(x) || x == -999999) ? missing : x) => :beds_icu,
);
data.beds_acute = data.beds_allbeds - data.beds_icu
sort!(data, [:hospital, :date]);

In [4]:
start_date = Date(2020,  7, 1)
end_date   = Date(2020, 11, 1)
date_range = collect(start_date : Day(1) : end_date);

In [5]:
function mean_capacity(xs, dates)
    xs = [x for (x,d) in zip(xs,dates) if d in date_range]
    if all(ismissing.(xs))
        return 0
    else
        xs = filter(x -> !ismissing(x), xs)
        return round(Int, sum(xs) / length(xs))
    end
end;

function max_capacity(xs)
    if all(ismissing.(xs))
        return 0
    else
        return maximum(skipmissing(xs))
    end
end;

function min_capacity(xs)
    if all(ismissing.(xs))
        return 0
    else
        return minimum(skipmissing(xs))
    end
end;

In [6]:
capacity_data = combine(groupby(data, [:hospital, :hospital_id]), [
    :beds_icu => min_capacity => :beds_icu_min,
    :beds_acute => min_capacity => :beds_acute_min,
    :beds_allbeds => min_capacity => :beds_allbeds_min,

    (:beds_icu, :date) => mean_capacity => :beds_icu_mean,
    (:beds_acute, :date) => mean_capacity => :beds_acute_mean,
    (:beds_allbeds, :date) => mean_capacity => :beds_allbeds_mean,
        
    :beds_icu => max_capacity => :beds_icu_max,
    :beds_acute => max_capacity => :beds_acute_max,
    :beds_allbeds => max_capacity => :beds_allbeds_max,
]);

In [7]:
capacity_names_icu = [:beds_icu_min, :beds_icu_mean, :beds_icu_max];
capacity_names_acute = [:beds_acute_min, :beds_acute_mean, :beds_acute_max];
capacity_names_allbeds = [:beds_allbeds_min, :beds_allbeds_mean, :beds_allbeds_max];

In [8]:
filter!(row -> row.beds_icu_max + row.beds_acute_max + row.beds_allbeds_max > 0, capacity_data);

In [9]:
sort!(capacity_data, [:hospital, :hospital_id]);

In [10]:
hospitals = collect(zip(capacity_data.hospital, capacity_data.hospital_id))
N = length(hospitals)
@show N;
@show hospitals;

N = 4838


hospitals = [("Abbeville Area Medical Center", "421301"), ("Abbeville General Hospital", "190034"), ("Abbott Northwestern Hospital", "240057"), ("Abilene Regional Medical Center", "450558"), ("Abington Memorial Hospital", "390231"), ("Abraham Lincoln Memorial Hospital", "141322"), ("Abrazo Arrowhead Campus", "030094"), ("Abrazo Central Campus", "030030"), ("Abrazo Scottsdale Campus", "030083"), ("Abrazo West Campus", "030110"), ("Abrom Kaplan Memorial Hospital", "191322"), ("Acadia General Hospital", "190044"), ("Acadia St Landry", "191319"), ("Acadian Medical Center", "190318"), ("Acmh Hospital", "390163"), ("Acuity Specialty Hospital Of Morgantown", "510092"), ("Acuity Specialty Hospital Of New Jersey", "312023"), ("Acuity Specialty Hospital Of Southern New Jersey", "312022"), ("Acuity Specialty Hospital Ohio Valley", "512005"), ("Ad Hospital East, Llc", "670102"), ("Adair County Memorial Hospital", "161310"), ("Adams County Regional Medical Center", "361326"), ("Adams Memorial Hospi

028"), ("North Houston Surgical Hospital Llc", "670280"), ("North Kansas City Hospital", "260096"), ("North Logan Mercy Hospital", "041300"), ("North Memorial Health", "240001"), ("North Mississippi Medical Center", "250004"), ("North Mississippi Medical Center-Gilmore Amory", "250025"), ("North Mississippi Medical Center-West Point", "250067"), ("North Oaks Medical Center, L L C", "190015"), ("North Okaloosa Medical Center", "100122"), ("North Ottawa Community Health System", "230174"), ("North Runnels Hospital", "451315"), ("North Shore Health", "241317"), ("North Shore Medical Center", "100029"), ("North Shore Medical Center -", "220035"), ("North Shore University Hospital", "330106"), ("North Suburban Medical Center", "060065"), ("North Sunflower Medical Center Cah", "251318"), ("North Texas Medical Center", "450090"), ("North Valley Health Center", "241337"), ("North Valley Hospital", "271336"), ("North Valley Hospital", "501321"), ("North Vista Hospital", "290005"), ("Northbay Me

orek Memorial Hospital", "140115"), ("Three Rivers Health", "230015"), ("Three Rivers Hospital", "441303"), ("Three Rivers Hospital", "501324"), ("Three Rivers Medical Center", "180128"), ("Throckmorton County Memorial Hospital", "451339"), ("Tidalhealth Peninsula Regional, Inc", "210019"), ("Tidelands Health", "420020"), ("Tidelands Waccamaw Community Hospital", "420098"), ("Tift Regional Medical Center", "110095"), ("Timpanogos Regional Hospital", "460052"), ("Tioga Medical Center", "351300"), ("Tippah County Hospital", "251337"), ("Tishomingo Health Services Inc", "250002"), ("Titus Regional Medical Center", "450080"), ("Titusville Hospital", "391314"), ("Tmc- Bonham Hospital", "451370"), ("Toledo Hospital The", "360068"), ("Tomah Memorial Hospital", "521320"), ("Tops Surgical Specialty Hospital", "450774"), ("Torrance Memorial Medical Center", "050351"), ("Touchette Regional Hospital Inc", "140077"), ("Touro Infirmary", "190046"), ("Towner County Medical Center", "351331"), ("Trace

In [11]:
capacity_data

,hospital,hospital_id,beds_icu_min,beds_acute_min,beds_allbeds_min,beds_icu_mean,beds_acute_mean,beds_allbeds_mean,beds_icu_max,beds_acute_max,beds_allbeds_max
,String,String,Float64,Float64,Float64,Int64,Int64,Int64,Float64,Float64,Float64
1,Abbeville Area Medical Center,421301,0.0,10.6,10.6,0,16,16,0.0,18.0,18.0
2,Abbeville General Hospital,190034,6.4,27.0,34.0,7,31,38,7.0,31.0,38.0
3,Abbott Northwestern Hospital,240057,42.4,376.2,429.3,51,614,665,63.3,642.4,693.1
4,Abilene Regional Medical Center,450558,9.3,51.4,66.0,14,68,82,21.3,83.1,104.4
5,Abington Memorial Hospital,390231,89.0,399.0,488.0,89,432,521,89.0,436.0,525.0
6,Abraham Lincoln Memorial Hospital,141322,0.0,25.0,25.0,0,25,25,0.0,25.0,25.0
7,Abrazo Arrowhead Campus,030094,38.0,125.0,164.8,39,145,187,59.3,192.3,251.6
8,Abrazo Central Campus,030030,23.6,76.3,104.3,29,81,109,34.0,97.2,125.3
9,Abrazo Scottsdale Campus,030083,12.3,37.7,51.7,14,46,60,14.0,55.1,69.1


In [12]:
capacity_data_long = stack(capacity_data, Not([:hospital, :hospital_id]), variable_name=:capacity_name, value_name=:capacity_value);

In [13]:
capacity_data_long_icu = filter(row -> Symbol(row.capacity_name) in capacity_names_icu, capacity_data_long);
capacity_data_long_acute = filter(row -> Symbol(row.capacity_name) in capacity_names_acute, capacity_data_long);
capacity_data_long_allbeds = filter(row -> Symbol(row.capacity_name) in capacity_names_allbeds, capacity_data_long);

In [14]:
capacity_data_output = select(capacity_data, :hospital, :hospital_id, :beds_icu_mean => :capacity_icu, :beds_acute_mean => :capacity_acute, :beds_allbeds_mean => :capacity_allbeds);

In [15]:
capacity_data_output |> CSV.write("../data/capacity_hhs.csv");